In [99]:
using CSV
using DataFrames
using LinearAlgebra


network_df = CSV.read("../SiouxFalls/SiouxFalls_net.txt", DataFrame; delim='\t')
network_df = select(network_df, Not(:Column1))
node_df = CSV.read("../SiouxFalls/SiouxFalls_node.txt", DataFrame; delim='\t')
node_df = select(node_df, Not(";"))
od_df = CSV.read("../SiouxFalls/SiouxFalls_od.csv", DataFrame)

ods = [(row.O, row.D) for row in eachrow(od_df)] 
road_link = [(row.init_node, row.term_node, row.length) for row in eachrow(network_df)]
A = [(row.init_node, row.term_node) for row in eachrow(network_df)] #arcs


76-element Vector{Tuple{Int64, Int64}}:
 (1, 2)
 (1, 3)
 (2, 1)
 (2, 6)
 (3, 1)
 (3, 4)
 (3, 12)
 (4, 3)
 (4, 5)
 (4, 11)
 (5, 4)
 (5, 6)
 (5, 9)
 ⋮
 (21, 22)
 (21, 24)
 (22, 15)
 (22, 20)
 (22, 21)
 (22, 23)
 (23, 14)
 (23, 22)
 (23, 24)
 (24, 13)
 (24, 21)
 (24, 23)

In [103]:
O = unique(od_df[!, :O])
D = unique(od_df[!, :D])

24-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
 15
 16
 17
 18
 19
 20
 21
 22
 23
 24

Row,O,D,Ton
,Int64,Int64,Float64
1,2,1,100.0
2,3,1,100.0
3,4,1,500.0
4,5,1,200.0
5,6,1,300.0
6,7,1,500.0
7,8,1,800.0
8,9,1,500.0
9,10,1,1300.0


In [126]:
alpha = 0.15
beta = 4
t0_a = Dict((row.init_node, row.term_node) => row.free_flow_time for row in eachrow(network_df))
c_a = Dict((row.init_node, row.term_node) => row.capacity for row in eachrow(network_df))

d_od = Dict((row.O, row.D) => row.Ton for row in eachrow(od_df))


n_routes = 3


3

In [130]:
using Pandas: read_pickle
P = read_pickle("../SiouxFalls/OD_route.pickle")

Dict{Any, Any} with 528 entries:
  (18, 16) => Dict{Any, Any}(0=>[18, 16])
  (16, 14) => Dict{Any, Any}(0=>[16, 17, 19, 15, 14], 1=>[16, 10, 11, 14])
  (11, 17) => Dict{Any, Any}(0=>[11, 10, 16, 17], 2=>[11, 10, 17], 1=>[11, 14, …
  (24, 15) => Dict{Any, Any}(0=>[24, 21, 22, 15], 2=>[24, 23, 14, 15], 1=>[24, …
  (17, 12) => Dict{Any, Any}(0=>[17, 16, 10, 11, 12], 2=>[17, 16, 8, 6, 5, 4, 3…
  (8, 15)  => Dict{Any, Any}(0=>[8, 16, 17, 19, 15], 2=>[8, 16, 10, 15], 1=>[8,…
  (19, 14) => Dict{Any, Any}(0=>[19, 15, 14], 1=>[19, 17, 16, 10, 11, 14])
  (7, 18)  => Dict{Any, Any}(0=>[7, 18])
  (7, 8)   => Dict{Any, Any}(0=>[7, 8])
  (14, 15) => Dict{Any, Any}(0=>[14, 15], 1=>[14, 23, 22, 15])
  (15, 23) => Dict{Any, Any}(0=>[15, 22, 23], 1=>[15, 14, 23])
  (1, 9)   => Dict{Any, Any}(0=>[1, 3, 4, 5, 9], 2=>[1, 3, 12, 11, 10, 9], 1=>[…
  (19, 16) => Dict{Any, Any}(0=>[19, 17, 16])
  (22, 9)  => Dict{Any, Any}(0=>[22, 15, 10, 9], 1=>[22, 20, 18, 16, 10, 9])
  (7, 24)  => Dict{Any, Any}(0=>[7, 18, 

In [131]:
py"""
def is_continuous_subsequence(my_tuple, my_list):
    tuple_length = len(my_tuple)
    list_length = len(my_list)
    
    for i in range(0, list_length - tuple_length + 1):
        if tuple(my_list[i:i+tuple_length]) == my_tuple:
            return True
    return False

"""

# Access the Python function as if it were a Julia function
is_continuous_subsequence = py"is_continuous_subsequence"


PyObject <function is_continuous_subsequence at 0x7f3857b80560>

In [148]:
using JuMP
using MosekTools


# Model initialization
model = Model(Mosek.Optimizer)

# Variables
@variable(model, x_a[a in A] >= 0)
@variable(model, mu_a[a in A] >= 0)
@variable(model, theta_a[a in A] >= 0)
@variable(model, u_a[a in A] >= 0)

@variable(model, v_p[(od, p) in [(od, p) for od in ods for p in keys(P[od])]] >= 0)


# Objective function
@objective(model, Min, sum(t0_a[a] * x_a[a] + (t0_a[a] * alpha / (beta+1)/ c_a[a]) * mu_a[a] for a in A))

# # Constraint: Sum of v_p over paths in P_{od} equals 1 for each origin-destination pair
@constraint(model, [od in ods], sum(v_p[(od, p)] for p in keys(P[od])) == 1)

# Constraint: x_a equals the sum of d_{od} * v_p over all o, d, and paths that include arc a
@constraint(model, [a in A], x_a[a] == sum(d_od[od] * v_p[(od, p)] for od in ods for p in keys(P[od]) if is_continuous_subsequence(a,P[od][p])))


# Constraints
for a in A
    @constraint(model, [2 * x_a[a], theta_a[a] - 1, theta_a[a] + 1] in SecondOrderCone())
    @constraint(model, [2 * theta_a[a], u_a[a] - x_a[a], u_a[a] + x_a[a]] in SecondOrderCone())
    @constraint(model, [2 * u_a[a], mu_a[a] - x_a[a], mu_a[a] + x_a[a]] in SecondOrderCone())
end

# Solve the model
optimize!(model)

# Check the status of the solution
status = termination_status(model)

if status == MOI.OPTIMAL
    # Extracting the solution
    x_a_solution = value.(x_a)
    mu_a_solution = value.(mu_a)
    theta_a_solution = value.(theta_a)
    u_a_solution = value.(u_a)
    v_p_solution = value.(v_p)
    println("Optimal solution found")
    # You can print or use the solutions as needed
else
    println("Problem status: $status")
end


Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 604             
  Affine conic cons.     : 228 (684 rows)
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 1455            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 241
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - primal attempts        : 1                 successes    